In [9]:
import tensorflow as tf
import numpy as np
import math

In [10]:
path = tf.keras.utils.get_file('shakespeare.txt', 'https://homl.info/shakespeare')

In [11]:
with open(path) as o:
    Text = o.read()

In [12]:
print(Text[:15])

First Citizen:



In [13]:
Token = tf.keras.preprocessing.text.Tokenizer(char_level = True)
Token.fit_on_texts([Text])

In [15]:
dist_char = Token.word_index
n_distC = len(dist_char)
print("Distinct characters: ", dist_char)
print("-----------------------------------------------")
print("Total Distinct characters: ", n_distC)
print("-----------------------------------------------")


Distinct characters:  {' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'i': 6, 'h': 7, 's': 8, 'r': 9, 'n': 10, '\n': 11, 'l': 12, 'd': 13, 'u': 14, 'm': 15, 'y': 16, 'w': 17, ',': 18, 'c': 19, 'f': 20, 'g': 21, 'b': 22, 'p': 23, ':': 24, 'k': 25, 'v': 26, '.': 27, "'": 28, ';': 29, '?': 30, '!': 31, '-': 32, 'j': 33, 'q': 34, 'x': 35, 'z': 36, '3': 37, '&': 38, '$': 39}
-----------------------------------------------
Total Distinct characters:  39
-----------------------------------------------


In [8]:
Token.texts_to_sequences("quantum physics")

[[34],
 [14],
 [5],
 [10],
 [3],
 [14],
 [15],
 [1],
 [23],
 [7],
 [16],
 [8],
 [6],
 [19],
 [8]]

In [9]:
Token.sequences_to_texts([[34],
 [14],
 [5],
 [10],
 [3],
 [14],
 [15],
 [1],
 [23],
 [7],
 [16],
 [8],
 [6],
 [19],
 [8]])

['q', 'u', 'a', 'n', 't', 'u', 'm', ' ', 'p', 'h', 'y', 's', 'i', 'c', 's']

In [10]:
[text_encoded] = np.array(Token.texts_to_sequences([Text])) - 1

In [11]:
print(Text[:15])
print(text_encoded[:15])

First Citizen:

[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10]


In [12]:
text_size = len(Text)
print(text_size)

1115394


In [13]:
train_size = int(text_size * 0.80)
print(train_size)
train_dataset = tf.data.Dataset.from_tensor_slices(text_encoded[:train_size])
val_dataset = tf.data.Dataset.from_tensor_slices(text_encoded[train_size:])
print(train_dataset)

892315
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>


In [14]:
n_sequences = 100
Window_size = n_sequences + 1
train_dataset = train_dataset.window(Window_size, shift=1, drop_remainder=True)
val_dataset = val_dataset.window(Window_size, shift=1, drop_remainder=True)
print(train_dataset)
train_dataset = train_dataset.flat_map(lambda w: w.batch(Window_size))
val_dataset = val_dataset.flat_map(lambda w: w.batch(Window_size))
print(train_dataset)

<_WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([]))>
<_FlatMapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>


In [15]:
batch_size = 32
train_dataset = train_dataset.shuffle(20000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
print(train_dataset)
train_dataset = train_dataset.map(lambda w: (w[:, :-1], w[:, 1:]))
train_dataset = train_dataset.map(lambda X, Y: (tf.one_hot(X, depth=n_distC), Y))
val_dataset = val_dataset.map(lambda w: (w[:, :-1], w[:, 1:]))
val_dataset = val_dataset.map(lambda X, Y: (tf.one_hot(X, depth=n_distC), Y))
print(train_dataset)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


<_BatchDataset element_spec=TensorSpec(shape=(None, None), dtype=tf.int32, name=None)>
<_MapDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>


In [16]:
num_batches = sum(1 for _ in train_dataset)
print("Número total de batches no train_dataset:", num_batches)

Número total de batches no train_dataset: 27882


In [17]:
model = tf.keras.Sequential([
        tf.keras.layers.GRU(128, return_sequences=True, dropout=0.2, input_shape=[None, n_distC]),
        tf.keras.layers.GRU(128, return_sequences=True, dropout=0.2,),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_distC, activation='softmax'))
])

C:\Users\marce\anaconda4\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('model.keras', monitor='accuracy', save_best_only=False, mode='max', verbose=1, save_freq=100)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=5,    
    restore_best_weights=True
)
backup_restore_callback = tf.keras.callbacks.BackupAndRestore(
    backup_dir='./backups', 
    save_freq=50
)

In [19]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Nadam(), metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[early_stopping, checkpoint_callback, backup_restore_callback] )

In [16]:
model = tf.keras.models.load_model('model.keras')

In [17]:
def preprocess(texts):
    X = np.array(Token.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, n_distC)

In [18]:
X_new = preprocess(["Just nlp and not Sk"])
Y_pred = np.argmax(model(X_new), axis=-1)
Token.sequences_to_texts(Y_pred + 1)[0][-1]

'i'

In [19]:
def next_char(text, risk = 1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    y_proba_rescaled = np.log(y_proba) / risk
    char_id = tf.random.categorical(y_proba_rescaled, num_samples=1) + 1
    return Token.sequences_to_texts(char_id.numpy())[0]

In [20]:
def create_text(text, n_chars=80, risk=1):
     for _ in range(n_chars):
         text += next_char(text, risk)
     return text

In [21]:
print(create_text("thy", n_chars=20, risk=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
thy mother, so so much 


In [ ]:
#%pip install transformers
#%pip install torch
#%pip install tf-keras

In [ ]:
from transformers import TFOpenAIGPTLMHeadModel, OpenAIGPTTokenizer
model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
text = "Skynet will"
inputs = tokenizer.encode(text, return_tensors="tf") 
output = model.generate(inputs, max_length=50, num_return_sequences=1, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


In [63]:
print(generated_text)

skynet will be able to get to the bottom of this . " 
 " i 'm not sure i can do that , " said the captain . " i 'm not sure i can even get to the bottom of this . " 
 " i 'm
